In [1]:
import sys
sys.path.append("/home/code")

# Dataset

In [1]:
from database import *

datasetdb = DatasetDB()
datasetdb.get_df()

,id,task,size
0,sangmu_str_MH,[STR],501
0,sangmu_str_DI,[STR],568
0,sangmu_str_DH,[STR],566
0,sangmu_std_MH,[STD],501
0,sangmu_std_DI,[STD],568
0,sangmu_std_DH,[STD],566
0,sangmu_str_MH_v2,[STR],1491
0,sangmu_str_DI_v2,[STR],1840
0,sangmu_str_DH_v2,[STR],2208
0,aihub_rec,[STR],884070


In [10]:
datasetdb.get_all_id()

In [16]:
from pathlib import Path
from PIL import Image

datasetdb = DatasetDB()
all_ids = datasetdb.get_all_id()
for id in all_ids:
    labels = datasetdb.get_all_labels(id)
    path_list = [Path(f"/home/datasets/{id}")/(label.split("\t")[0]) for label in labels]

    vn, sn, hn = 0, 0, 0
    for path in path_list:
        image = Image.open(path)
        w, h = image.size
        if h > w*2: # vertical
            vn += 1
        elif h*2 > w: # vertical
            sn += 1
        else: # horizental
            hn += 1
    print(id, vn, sn, hn)    

sangmu_str_MH 71 140 290
sangmu_str_DI 74 158 336
sangmu_str_DH 48 140 378
sangmu_std_MH 118 114 269
sangmu_std_DI 115 147 306
sangmu_std_DH 95 125 346
outsourcing1_str 77 1226 4003
outsourcing1(exception)_str 67 528 1197
sangmu_str_MH_v2 142 583 766
sangmu_str_DI_v2 147 622 1071
sangmu_str_DH_v2 120 641 1447
aihub_rec 76767 290853 516450
rec_test 0 0 3
rec_SVTP 13 287 345
rec_SVT 0 128 519
rec_IIIT5K 7 1820 3173
rec_ic15 55 988 1034
rec_ic13 0 150 707
rec_ic03 1 117 742
rec_CUTE80 1 129 158
outsourcing_v2.1_det 63 114 805
outsourcing_v2.1(exception)_det 19 32 123
outsourcing_v2.2_det 118 310 1402
outsourcing_v2.2(exception)_det 29 63 216
outsourcing_v2.1_rec 99 558 2751
outsourcing_v2.1(exception)_rec 27 113 456
outsourcing_v2.2_rec 189 881 4462
outsourcing_v2.2(exception)_rec 33 186 739


In [4]:
from database import *

datasetdb = DatasetDB()
datasetdb.get_df()

,id,task,size
0,sangmu_str_MH,[STR],501
0,sangmu_str_DI,[STR],568
0,sangmu_str_DH,[STR],566
0,sangmu_std_MH,[STD],501
0,sangmu_std_DI,[STD],568
0,sangmu_std_DH,[STD],566
0,outsourcing1_str,[STR],5306
0,outsourcing1(exception)_str,[STR],1792
0,sangmu_str_MH_v2,[STR],1491
0,sangmu_str_DI_v2,[STR],1840


NameError: name 'datasetdb' is not defined

# Labelset

In [8]:
from database import *
datadb = DatasetDB()
labeldb = LabelsetDB()
labeldb.get_all_id()

['sangmu_str',
 'sangmu_std',
 'sangmu_and_outsourcing_str',
 'outsourcing_str',
 'sangmu_and_outsourcing_std',
 'outsourcing_std',
 'sangmu_str_v2',
 'sangmu_and_outsourcing_str_v2',
 'full_dataset_str_v1',
 'rec_test']

In [1]:
from database import *
datadb = DatasetDB()
datadb.relative_to("id", "path", "absolute")


'E:/workspace/paddleocr/datasets/id/path'

In [ ]:
from pathlib import Path
x = "./hello/world/text.txt"
Path("./test/")/x

WindowsPath('test/hello/world/text.txt')

#### 레이블 셋 생성

In [2]:
from database import *
datadb = DatasetDB()
labeldb = LabelsetDB()
datadb.get_all_id()

# labeldb.make("sangmu_std", ["sangmu_std_MH", "sangmu_std_DI", "sangmu_std_DH"], split_ratio=[8, 1, 1], random_seed=100)

oursourcing_datasets = [
    # "aihub_rec",
    "outsourcing_v1_rec",
    "outsourcing_v1(exception)_rec",
    "outsourcing_v2.1_rec",
    "outsourcing_v2.1(exception)_rec",
    "outsourcing_v2.2_rec",
    "outsourcing_v2.2(exception)_rec",
    "outsourcing_v2.3_rec",
    "outsourcing_v2.3(exception)_rec",
    "outsourcing_v2.4_rec",
    "outsourcing_v2.4(exception)_rec",
    "outsourcing_v2.5_rec",
    "outsourcing_v2.5(exception)_rec",
    "outsourcing_v2.6.1_rec",
    "outsourcing_v2.6.1(exception)_rec",
    "outsourcing_v2.6.2_rec",
    "outsourcing_v2.6.2(exception)_rec",
    # "rec_CUTE80",
    # "rec_ic03",
    # "rec_ic13",
    # "rec_ic15",
    # "rec_IIIT5K",
    # "rec_SVT",
    # "rec_SVTP"
]

labeldb.make("big_train_and_small_sangmu_eval", oursourcing_datasets, [1, 0, 0], random_seed=100, individual_split_ratio={
    "sangmu_str_MH_v2":[0, 1, 0],
    "sangmu_str_DI_v2":[0, 1, 0],
    "sangmu_str_DH_v2":[0, 1, 0]
})


# labeldb.make("sangmu_str_v3", ["sangmu_str_MH_v2", "sangmu_str_DI_v2", "sangmu_str_DH_v2"], split_ratio=[0, 0, 1], random_seed=100)
# labeldb.make("sangmu_str_v3", [], [8, 2, 0], individual_split_ratio={
#     "sangmu_str_DH_v2":[0, 0, 1],
#     "sangmu_str_DI_v2":[0, 0, 1],
#     "sangmu_str_MH_v2":[0, 0, 1]
#     }, random_seed=100)

# labeldb.make("outsourcing_std", ["outsourcing1_std", "outsourcing1(exception)_std"], [8, 1, 1], random_seed=100)
# labeldb.make("outsourcing_str", ["outsourcing1_str", "outsourcing1(exception)_str"], [8, 1, 1], random_seed=100)

# labeldb.make("direct_collection_v1", ["outsourcing_v2.1_rec", "outsourcing_v2.1(exception)_rec", "outsourcing_v2.2_rec", "outsourcing_v2.2(exception)_rec", "outsourcing1_str", "outsourcing1(exception)_str"], [8, 2, 0], individual_split_ratio={
#     "sangmu_str_DH_v2":[0, 0, 1],
#     "sangmu_str_DI_v2":[0, 0, 1],
#     "sangmu_str_MH_v2":[0, 0, 1]
#     }, random_seed=100)

# labeldb.make("rec_test", ["rec_test"], [1, 1, 1])

# labeldb.make("sangmu_and_outsourcing_str_v2", ["outsourcing1_str", "outsourcing1(exception)_str"], [8, 2, 0], individual_split_ratio={
#     "sangmu_str_MH_v2":[0, 0, 1],
#     "sangmu_str_DI_v2":[0, 0, 1],
#     "sangmu_str_DH_v2":[0, 0, 1]
#     }, random_seed=100)

outsourcing_v1_rec [1, 0, 0]
outsourcing_v1(exception)_rec [1, 0, 0]
outsourcing_v2.1_rec [1, 0, 0]
outsourcing_v2.1(exception)_rec [1, 0, 0]
outsourcing_v2.2_rec [1, 0, 0]
outsourcing_v2.2(exception)_rec [1, 0, 0]
outsourcing_v2.3_rec [1, 0, 0]
outsourcing_v2.3(exception)_rec [1, 0, 0]
outsourcing_v2.4_rec [1, 0, 0]
outsourcing_v2.4(exception)_rec [1, 0, 0]
outsourcing_v2.5_rec [1, 0, 0]
outsourcing_v2.5(exception)_rec [1, 0, 0]
outsourcing_v2.6.1_rec [1, 0, 0]
outsourcing_v2.6.1(exception)_rec [1, 0, 0]
outsourcing_v2.6.2_rec [1, 0, 0]
outsourcing_v2.6.2(exception)_rec [1, 0, 0]
sangmu_str_MH_v2 [0, 1, 0]
sangmu_str_DI_v2 [0, 1, 0]
sangmu_str_DH_v2 [0, 1, 0]
2. save label files
3. make infer label files


#### k_fold 레이블 셋 생성

In [1]:
from database import *
labeldb = LabelsetDB()
labeldb.make_k_fold(["sangmu_and_outsourcing_rec_v3"], "sangmu_and_outsourcing_rec_v3_k_fold", k=5)
# labeldb.get_config("sangmu_and_outsourcing_rec_v3_k_fold_5_1")

1. Load all label files
2. split labels in k segments
3. save
labelsets/sangmu_and_outsourcing_rec_v3_k_fold_5_1/train_label.txt


NameError: name 'exit' is not defined

##### Model

In [1]:
from database import *
modeldb = ModelDB()
import os
# modeldb.get_config("en_PP-OCRv3_det")
modeldb.get_all_id()
modeldb.get_config("en_PP-OCRv3_det", relative_to="absolute")
# for id in modeldb.get_all_id():
#     print(id)
#     modeldb.make_inference_model(id)

{'inference_model': 'E:/workspace/paddleocr/models/en_PP-OCRv3_det/inference_model/Student/inference.pdmodel',
 'inference_model_dir': 'E:/workspace/paddleocr/models/en_PP-OCRv3_det/inference_model/Student',
 'inference_model_weight': 'E:/workspace/paddleocr/models/en_PP-OCRv3_det/inference_model/Student/inference.pdiparams',
 'pretrained_model_dir': 'E:/workspace/paddleocr/models/en_PP-OCRv3_det/pretrained_model',
 'pretrained_model_weight': 'E:/workspace/paddleocr/models/en_PP-OCRv3_det/pretrained_model/pretrained.pdparams',
 'train_config': 'E:/workspace/paddleocr/models/en_PP-OCRv3_det/pretrained_model/config.yml'}

# Work

In [2]:
from database import *
workdb = WorkDB()
workdb.get_df()

,id,labelsets,model,trained_epoch
0,det_PPOCR_sangmu,[sangmu_std],ml_PP-OCRv3_det,200
0,det_PPOCR_outsourcing,[outsourcing_std],ml_PP-OCRv3_det,200
0,det_PPOCR_sangmu_and_outsourcing,[sangmu_and_outsourcing_std],ml_PP-OCRv3_det,200
0,rec_PPOCR_sangmu_v2,[sangmu_str_v2],korean_PP-OCRv3_rec,200
0,rec_PPOCR_sangmu_and_outsourcing_v2,[sangmu_and_outsourcing_str_v2],korean_PP-OCRv3_rec,200
0,rec_PPOCR_sangmu_v1,[sangmu_str],korean_PP-OCRv3_rec,200
0,rec_PPOCR_sangmu_and_outsourcing_v1,[sangmu_and_outsourcing_str],korean_PP-OCRv3_rec,200
0,rec_PPOCR_outsourcing_v1,[outsourcing_str],korean_PP-OCRv3_rec,200
0,rec_PPOCR_full_v1,[full_dataset_str_v1],korean_PP-OCRv3_rec,77
0,rec_test,[rec_test],korean_PP-OCRv3_rec,0


#### make

In [3]:
from database import *
workdb = WorkDB()
# workdb.make("det_PPOCR_sangmu", ["sangmu_std"], "ml_PP-OCRv3_det")
# workdb.make("det_PPOCR_outsourcing", ["outsourcing_std"], "ml_PP-OCRv3_det")
# workdb.make("det_PPOCR_sangmu_and_outsourcing", ["sangmu_and_outsourcing_std"], "ml_PP-OCRv3_det")

# workdb.make("rec_PPOCR_sangmu", ["sangmu_str"], "korean_PP-OCRv3_rec")
# workdb.make("rec_PPOCR_outsourcing", ["outsourcing_str"], "korean_PP-OCRv3_rec")
# workdb.make("rec_PPOCR_sangmu_and_outsourcing", ["sangmu_and_outsourcing_str"], "korean_PP-OCRv3_rec")


# workdb.make("rec_PPOCR_sangmu_v2", ["sangmu_str_v2"], "korean_PP-OCRv3_rec")
# workdb.make("rec_PPOCR_sangmu_and_outsourcing_v2", ["sangmu_and_outsourcing_str_v2"], "korean_PP-OCRv3_rec")
# workdb.make("det_PPOCR_sangmu_and_outsourcing_final_v1", ["sangmu_and_outsourcing_det_v3"], "ml_PP-OCRv3_det")
# workdb.make("rec_PPOCR_sangmu_and_outsourcing_final_v1", ["sangmu_and_outsourcing_rec_v3"], "korean_PP-OCRv3_rec_rotation_pretrained")
workdb.make("rec_PPOCR_sangmu_and_outsourcing_final_v3", ["big_train_and_small_sangmu_eval"], "korean_PP-OCRv3_rec_rotation_pretrained_v2")
workdb.make("rec_PPOCR_sangmu_and_outsourcing_final_v3_with_eng", ["big_train_and_small_sangmu_eval_with_eng"], "korean_PP-OCRv3_rec_rotation_pretrained_v2")

#### Work Infer

In [4]:
from database import *
workdb = WorkDB()
# workdb.infer("rec_PPOCR_full_auto_rotate_v2", "best", "test", data_dir="/home/resources/rotation_test_images/", labelsets=["/home/resources/rotation_test_images/rec_infer.txt"])
# workdb.infer("rec_rotation_pretrained_and_direct_collection_finetuning", "best", "test", data_dir="/home/datasets", labelsets=["/home/labelsets/full_dataset_str_v1/square_test_label_english_infer.txt"])
# workdb.infer("rec_PPOCR_full_auto_rotate_v2", "best", "test", data_dir="/home/datasets", labelsets=["/home/labelsets/full_dataset_str_v1/square_test_label_english_infer.txt"])
# workdb.infer("rec_PPOCR_sangmu_and_outsourcing_final_v2", "best", "test", data_dir="/home/resources/상무", labelsets=["/home/resources/상무/infer.txt"], save_path="/home/resources/상무/infer_result.txt")
# workdb.infer("rec_PPOCR_sangmu_and_outsourcing_final_v2", "best", "test", data_dir="/home/resources/세종시", labelsets=["/home/resources/세종시/infer.txt"], save_path="/home/resources/세종시/infer_result.txt")
# workdb.infer("rec_PPOCR_sangmu_and_outsourcing_final_v2", "best", "test", data_dir="/home/resources/세종시추가", labelsets=["/home/resources/세종시추가/infer.txt"], save_path="/home/resources/세종시추가/infer_result.txt")
workdb.infer("rec_PPOCR_sangmu_and_outsourcing_final_v1", "best", "test", data_dir="/home/resources/240425_세종_정면_간판_레이블링_rec", labelsets=["/home/resources/240425_세종_정면_간판_레이블링_rec/infer.txt"], save_path="/home/research/상무_세종_샘플_실증/infer_result.txt")

# workdb.infer("rec_PPOCR_full_auto_rotate_v2", "best", "test")


python /home/code/PaddleOCR/tools/infer_rec.py -c /home/works/rec_PPOCR_sangmu_and_outsourcing_final_v2/train_config.yml -o Global.pretrained_model=/home/works/rec_PPOCR_sangmu_and_outsourcing_final_v2/trained_model/best_model/model Global.checkpoints=/home/works/rec_PPOCR_sangmu_and_outsourcing_final_v2/trained_model/best_model/model Global.save_model_dir=/home/works/rec_PPOCR_sangmu_and_outsourcing_final_v2/trained_model/best_model/model Global.save_res_path=/home/research/상무_세종_샘플_실증/infer_result.txt Infer.data_dir=/home/research/상무_세종_샘플_실증 Infer.infer_file_list=['/home/research/상무_세종_샘플_실증/infer.txt']


#### Work Train

In [5]:
from database import *
workdb = WorkDB()
modeldb = ModelDB()
labelsetdb = LabelsetDB()

works = ["rec_PPOCR_sangmu_and_outsourcing_final_v7_2"]

for work in works:
    workdb.train(work, "latest", 500, relative_to="absolute")

python /home/code/PaddleOCR/tools/train.py -c works/rec_PPOCR_sangmu_and_outsourcing_final_v7_2/train_config.yml -o Global.checkpoints=/home/works/rec_PPOCR_sangmu_and_outsourcing_final_v7_2/trained_model/latest Global.epoch_num=500 Global.save_model_dir=works/rec_PPOCR_sangmu_and_outsourcing_final_v7_2/trained_model Train.dataset.data_dir=./datasets Train.dataset.label_file_list=['labelsets/outsourcing_only_train/train_label.txt','labelsets/sangmu_rec_split_v1_2/train_label.txt'] Eval.dataset.data_dir=./datasets Eval.dataset.label_file_list=['labelsets/outsourcing_only_train/eval_label.txt','labelsets/sangmu_rec_split_v1_2/eval_label.txt']


#### Work Eval

In [9]:
from database import *
workdb = WorkDB()
workdb.get_df()

tasks = ["train", "eval"]
works = ["rec_PPOCR_sangmu_and_outsourcing_final_v7_2"]

epochs = sum([list(range(141+i, 200, 5)) for i in range(1, 5)], [])
for work in works:
    for epoch in epochs:
        for task in tasks:
            workdb.eval(work, version=epoch, task=task, relative_to="absolute", check_exist=False, )
            


python /home/code/PaddleOCR/tools/eval.py -c works/rec_PPOCR_sangmu_and_outsourcing_final_v7_2/train_config.yml -o Global.work_id=rec_PPOCR_sangmu_and_outsourcing_final_v7_2 Global.version=142 Global.eval_task=train Global.checkpoints=works/rec_PPOCR_sangmu_and_outsourcing_final_v7_2/trained_model/iter_epoch_142.pdparams Global.save_model_dir=works/rec_PPOCR_sangmu_and_outsourcing_final_v7_2/trained_model Eval.dataset.data_dir=./datasets Eval.dataset.label_file_list=['labelsets/outsourcing_only_train/train_label.txt','labelsets/sangmu_rec_split_v1_2/train_label.txt'] Eval.save=True Eval.check_exist=False
python /home/code/PaddleOCR/tools/eval.py -c works/rec_PPOCR_sangmu_and_outsourcing_final_v7_2/train_config.yml -o Global.work_id=rec_PPOCR_sangmu_and_outsourcing_final_v7_2 Global.version=142 Global.eval_task=eval Global.checkpoints=works/rec_PPOCR_sangmu_and_outsourcing_final_v7_2/trained_model/iter_epoch_142.pdparams Global.save_model_dir=works/rec_PPOCR_sangmu_and_outsourcing_final

In [9]:
from database import *
workdb = WorkDB()
workdb.eval(id="det_PPOCR_sangmu_and_outsourcing_final_v1", version="best", task=None, relative_to="absolute", check_exist=False, data_dir="/home/resources/240425_세종_정면_간판_레이블링_det", labelsets=["/home/resources/240425_세종_정면_간판_레이블링_det/label.txt"], save=False)
workdb.eval(id="rec_PPOCR_sangmu_and_outsourcing_final_v1", version="best", task=None, relative_to="absolute", check_exist=False, data_dir="/home/resources/240425_세종_정면_간판_레이블링_rec", labelsets=["/home/resources/240425_세종_정면_간판_레이블링_rec/label.txt"], save=False)
# workdb.eval(id="rec_PPOCR_sangmu_and_outsourcing_final_v2", version="best", task="test", relative_to="absolute", check_exist=False, save=False)


python /home/code/PaddleOCR/tools/eval.py -c works/det_PPOCR_sangmu_and_outsourcing_final_v1/train_config.yml -o Global.work_id=det_PPOCR_sangmu_and_outsourcing_final_v1 Global.version=best Global.eval_task=None Global.checkpoints=works/det_PPOCR_sangmu_and_outsourcing_final_v1/trained_model/best_model/model.pdparams Global.save_model_dir=works/det_PPOCR_sangmu_and_outsourcing_final_v1/trained_model Eval.dataset.data_dir=/home/resources/240425_세종_정면_간판_레이블링_det Eval.dataset.label_file_list=['/home/resources/240425_세종_정면_간판_레이블링_det/label.txt'] Eval.save=False Eval.check_exist=False
python /home/code/PaddleOCR/tools/eval.py -c works/rec_PPOCR_sangmu_and_outsourcing_final_v1/train_config.yml -o Global.work_id=rec_PPOCR_sangmu_and_outsourcing_final_v1 Global.version=best Global.eval_task=None Global.checkpoints=works/rec_PPOCR_sangmu_and_outsourcing_final_v1/trained_model/best_model/model.pdparams Global.save_model_dir=works/rec_PPOCR_sangmu_and_outsourcing_final_v1/trained_model Eval.dat

In [3]:
from database import *
workdb = WorkDB()

# works = ["rec_PPOCR_sangmu", "rec_PPOCR_outsourcing", "rec_PPOCR_sangmu_and_outsourcing"]
# works = ["det_PPOCR_sangmu", "det_PPOCR_outsourcing", "det_PPOCR_sangmu_and_outsourcing"]
tasks = ["train", "eval", "test"]
# epoch = range(1, 201, 1)

# works = ["det_PPOCR_sangmu", "det_PPOCR_outsourcing"]
# tasks = ["train", "eval", "test"]

# epochs = range(1, 101, 1)
# works = ["det_PPOCR_sangmu"]
# for work in works:
#     for epoch in epochs:
#         for task in tasks:
#             workdb.eval(work, version=epoch, task=task, relative_to="absolute")

# works = ["det_PPOCR_outsourcing"]
# epochs = range(1, 201, 1)
# for work in works:
#     for epoch in epochs:
#         for task in tasks:
#             workdb.eval(work, version=epoch, task=task, relative_to="absolute")


works = ["rec_PPOCR_outsourcing_vertical_v1"]
# , ""
# works = ["rec_PPOCR_sangmu_v2", "rec_PPOCR_sangmu_and_outsourcing_v2"]
tasks = ["train", "eval", "test"]

epochs = sum([list(range(i, 201, 5)) for i in range(1, 6)], [])
for work in works:
    for epoch in epochs:
        for task in tasks:
            workdb.eval(work, version=epoch, task=task, relative_to="absolute", check_exist=True)
            


# workdb.eval("det_PPOCR_oursourcing", "best", 10, relative_to="project") 
# workdb.eval("det_PPOCR_sangmu_and_oursourcing", "best", 10)

AssertionError: version should be 'best', 'latest', 'pretrained', or positive integer less than trained_epoch 0 but 1 is given

In [3]:
from database import *
db = DatasetDB()
db.get_df()

,id,task,size
0,sangmu_str_MH,[STR],501
0,sangmu_str_DI,[STR],568
0,sangmu_str_DH,[STR],566
0,sangmu_std_MH,[STD],501
0,sangmu_std_DI,[STD],568
0,sangmu_std_DH,[STD],566
0,outsourcing1_str,[STR],5373
0,outsourcing1(exception)_str,[STR],1792
0,sangmu_str_MH_v2,[STR],1579
0,sangmu_str_DI_v2,[STR],1946


### Make Inference Model

In [1]:
from database import *
workdb = WorkDB()
# workdb.export("det_PPOCR_sangmu_and_outsourcing", "best")
workdb.export("rec_PPOCR_sangmu_and_outsourcing_final_v1", "best")

python code/PaddleOCR/tools/export_model.py -c /home/works/rec_PPOCR_sangmu_and_outsourcing_final_v1/trained_model/config.yml -o Global.save_inference_dir=/home/works/rec_PPOCR_sangmu_and_outsourcing_final_v1/inference_models/inference_best Global.checkpoints=/home/works/rec_PPOCR_sangmu_and_outsourcing_final_v1/trained_model/best_model/model.pdparams


'/home/works/det_PPOCR_sangmu/inference_models/inference_best'

In [2]:
from database import *
workdb = WorkDB()
workdb.get_report_df("det_PPOCR_sangmu_old")

,version,step,acc,loss,precision,recall,dataset
0,latest,NaN,NaN,NaN,0.666667,1.000000,NaN
0,latest,NaN,NaN,NaN,0.666667,1.000000,NaN
0,latest,NaN,NaN,NaN,0.399777,0.656307,test


In [ ]:
from database import *
import project
from pathlib import Path
# config = "configs\det\det_mv3_db.yml"
# labelsets = ["test", "test_k_fold_5_1"]
# model = "MobileNetV3_large_x0_5"
# name = "test"
# WorkDB().make(name, labelsets, model, config)

WorkDB().train("test", 30)


In [8]:

from database import *
from pathlib import Path
import platform

def full_infer(image_dir, det_model_dir, rec_model_dir, save_dir):
    code = 'code/PaddleOCR/tools/infer/predict_system.py'
    det_algorithm="DB"

    # rec_model_dir = "E:/workspace/paddleocr/outputs/rec___korean_PP-OCRv3_rec___rec_08_02_90_random_k_fold_5_1___default_config___finetune/trained_model/inference_model"
    rec_char_dict_path = f"{project.PROJECT_ROOT}/code/PaddleOCR/ppocr/utils/dict/korean_dict.txt"
    os_name = platform.system()
    if os_name == "Windows":
        vis_font_path = "C:\Windows\Fonts\malgun.ttf"
    elif os_name == "Linux":
        vis_font_path = "/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf"

    command = f"""python {code} --image_dir={image_dir} \
    --det_model_dir={det_model_dir} \
    --det_algorithm={det_algorithm} \
    --rec_model_dir={rec_model_dir} \
    --rec_image_shape="3, 32, 100" \
    --rec_char_dict_path={rec_char_dict_path} \
    --vis_font_path={vis_font_path} \
    --draw_img_save_dir={save_dir} \
    --drop_score=0.3 \
    --det_db_box_thresh=0.4"""
    
    path = Path(project.PROJECT_ROOT)/"inference.sh"
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "a") as f:
        f.write(command+"\n")
    print(command)
    
modeldb = ModelDB()
workdb = WorkDB()

# det_model_dir = modeldb.get_config("MobileNetV3_large_x0_5")["inference_model_dir"] 
# rec_model_dir = modeldb.get_config("korean_PP-OCRv3_rec")["inference_model_dir"]
# save_dir = f"{project.PROJECT_ROOT}/inference/Mobile_PaddleOCR_pretrained"
# full_infer(det_model_dir, rec_model_dir, save_dir)

# image_dir = "./origin_datasets/sangmu_real_image_det(cropped)"
# det_model_dir = modeldb.get_config("ml_PP-OCRv3_det")["inference_model_dir"]
# rec_model_dir = modeldb.get_config("korean_PP-OCRv3_rec")["inference_model_dir"]
# save_dir = f"{project.PROJECT_ROOT}/inference/PaddleOCR_PaddleOCR_pretrained_sangmu"
# full_infer(image_dir, det_model_dir, rec_model_dir, save_dir)

# image_dir = "./origin_datasets/sangmu_real_image_det(cropped)"
# det_model_dir = workdb.get_config("det___ml_PP-OCRv3_det___ai_hub_det_08_02_90_random_k_fold_5_1___default_config___finetune")["inference_model_dir"]
# rec_model_dir = workdb.get_config("rec___korean_PP-OCRv3_rec___rec_08_02_90_random_k_fold_5_1___default_config___finetune")["inference_model_dir"]
# save_dir = f"{project.PROJECT_ROOT}/inference/PPOCR_PPOCR_tuned_sangmu"
# full_infer(image_dir, det_model_dir, rec_model_dir, save_dir)

# image_dir = "./datasets/ai_hub_korean_det_partial_test/images"
# det_model_dir = workdb.get_config("det___ml_PP-OCRv3_det___ai_hub_det_08_02_90_random_k_fold_5_1___default_config___finetune")["inference_model_dir"]
# rec_model_dir = workdb.get_config("rec___korean_PP-OCRv3_rec___rec_08_02_90_random_k_fold_5_1___default_config___finetune")["inference_model_dir"]
# save_dir = f"{project.PROJECT_ROOT}/inference/PPOCR_PPOCR_tuned_aihub"
# full_infer(image_dir, det_model_dir, rec_model_dir, save_dir)

from database import *
workdb = WorkDB()


image_dir = f"{project.PROJECT_ROOT}/resources/test_images"

image_dir = "/home/datasets/sangmu_std_MH/images"
image_dir = "/home/resources/test_images/"
image_dir = "./vertical"
image_dir = "/home/webtoon"
image_dir = "/home/resources/240425_세종_정면_간판_레이블링_det/1"

det_model_dir = workdb.get_inference_model("det_PPOCR_sangmu_and_outsourcing", "best", relative_to="absolute")
det_model_dir = f"{det_model_dir}/Student"

rec_model_dir = workdb.get_inference_model("rec_PPOCR_sangmu_and_outsourcing_final_v1", "best", relative_to="absolute")
# det_model_dir = f"{det_model_dir}/inference"

save_path = f"/home/resources/240425_세종_정면_간판_레이블링_result"
full_infer(image_dir, det_model_dir, rec_model_dir, save_path)

python code/PaddleOCR/tools/infer/predict_system.py --image_dir=/home/resources/240425_세종_정면_간판_레이블링_det/1     --det_model_dir=/home/works/det_PPOCR_sangmu_and_outsourcing/inference_models/inference_best/Student     --det_algorithm=DB     --rec_model_dir=/home/works/rec_PPOCR_sangmu_and_outsourcing_final_v1/inference_models/inference_best     --rec_image_shape="3, 32, 100"     --rec_char_dict_path=/home/code/PaddleOCR/ppocr/utils/dict/korean_dict.txt     --vis_font_path=/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf     --draw_img_save_dir=/home/resources/240425_세종_정면_간판_레이블링_result     --drop_score=0.3     --det_db_box_thresh=0.4


In [ ]:
from pathlib import Path

Path("./datasets/ai_hub_korean_image_det/train2/source/간판/돌출간판/돌출간판1/간판_돌출간판_001449.jpg").exists()

False

In [ ]:
with open("E:/workspace/paddleocr/datasets/ai_hub_korean_det/label2.txt") as f:
    lines = [line.rstrip("\n") for line in f.readlines()]

a, b, c = 0, 0, 0
for line in lines:
    if "가로형간판" in line:
        a+= 1
    elif "세로형간판" in line:
        b+= 1
    elif "돌출간판" in line:
        c+= 1
print(a, b, c)

80242 25621 55803


In [ ]:
with open("E:/workspace/paddleocr/datasets/ai_hub_korean_det/label3.txt", encoding="cp949") as f:
    with open("E:/workspace/paddleocr/datasets/ai_hub_korean_det/label2.txt", "w", encoding="utf-8") as f2:
        for line in f.readlines():
            f2.write(line)

In [ ]:
from dataset.aihub.hangul_real_image_dataset import *
dataset = HangulRealImageDataset()


*******Checked Dir List*******
datasets\ai_hub_korean_det\train1\source\간판\세로형간판
datasets\ai_hub_korean_det\val1\source\간판\세로형간판
datasets\ai_hub_korean_det\train2\source\간판\가로형간판\가로형간판1
datasets\ai_hub_korean_det\train2\source\간판\돌출간판\돌출간판1
datasets\ai_hub_korean_det\train2\source\간판\세로형간판
datasets\ai_hub_korean_det\val2\source\간판\세로형간판
******************************


In [ ]:
dataset = HangulRealImage_BoxDetectionDataset(dataset)

In [ ]:
x, y = dataset[0]
dataset.show_xy(x, y)
y

[{'bbox': [[762, 306], [849, 306], [849, 518], [762, 518]], 'label': '소망'},
 {'bbox': [[754, 547], [837, 547], [837, 900], [754, 900]], 'label': '미용실'},
 {'bbox': [[730, 1053], [845, 1053], [845, 1147], [730, 1147]],
  'label': 'xxx'},
 {'bbox': [[871, 1078], [1047, 1078], [1047, 1139], [871, 1139]],
  'label': 'xxx'},
 {'bbox': [[1177, 732], [1479, 732], [1479, 794], [1177, 794]],
  'label': 'xxx'},
 {'bbox': [[1429, 707], [1502, 707], [1502, 748], [1429, 748]],
  'label': 'xxx'},
 {'bbox': [[1342, 1075], [1428, 1075], [1428, 1160], [1342, 1160]],
  'label': 'xxx'},
 {'bbox': [[184, 345], [361, 345], [361, 611], [184, 611]], 'label': 'xxx'},
 {'bbox': [[121, 291], [184, 291], [184, 342], [121, 342]], 'label': 'xxx'},
 {'bbox': [[365, 194], [460, 194], [460, 252], [365, 252]], 'label': 'xxx'},
 {'bbox': [[391, 536], [443, 536], [443, 587], [391, 587]], 'label': 'xxx'},
 {'bbox': [[355, 610], [517, 610], [517, 684], [355, 684]], 'label': 'xxx'},
 {'bbox': [[172, 748], [240, 748], [240, 

In [ ]:
import pandas as pd
import numpy as np
df = pd.DataFrame({"epoch":[], "step":[], "acc":[], "loss":[], "precision":[], "recall":[]})

df = df._append({"epoch":1, "step":2, "acc":3, "loss":4, "precision":5, "recall":1}, ignore_index=True)
df.to_csv("test.csv")
df

,epoch,step,acc,loss,precision,recall
0,1.0,2.0,3.0,4.0,5.0,1.0


In [ ]:
df = pd.read_csv("test.csv", index_col=0)
df

,epoch,step,acc,loss,precision,recall
0,1.0,2.0,3.0,4.0,5.0,1.0


In [ ]:
from database import *

WorkDB().get_all_id()
WorkDB().get_config("det_MobileNet_sangmu")

{'id': 'det_MobileNet_sangmu',
 'labelsets': ['sangmu'],
 'model': 'MobileNetV3_large_x0_5',
 'pretrained': True,
 'train_config': './works/det_MobileNet_sangmu/train_config.yml',
 'result_path': './works/det_MobileNet_sangmu/result.csv'}

In [ ]:
from database import *

WorkDB().get_all_id()
# WorkDB().infer_det("det_MobileNet_sangmu")
# WorkDB().get_best_model_weight("det_MobileNet_sangmu")
# WorkDB().infer_det("det_MobileNet_sangmu", "test", "best")
WorkDB().infer_det("det_PPOCR_sangmu", "best", "test")

In [14]:
WorkDB().get_report_df("det_PPOCR_sangmu_and_outsourcing").groupby("task").count()

,work_id,version,precision,recall,hmean,fps
task,,,,,,
eval,200,200,200,200,200,200
test,200,200,200,200,200,200
train,200,200,200,200,200,200


In [1]:
1

1